**Introdução à Ciência de Dados - Trabalho 01**
**Extração de Dados**

Sérgio Garcia Barbosa Filho - 493746

**Tema/Site utilizado:** Catálogo de personagens do jogo Bungo Stray Dogs - Tales of The Lost / Mayoi Inu Kaikitan (Mobile)

**1) Importar bibliotecas necessárias**

Para este trabalho, utilizei as seguintes bibliotecas:
1. Requests: para fazer as requisições das páginas web;
2. BeatifulSoup4: para fazer a análise da página HTML e realizar a extração dos dados;
3. Pandas: para gravar gerar um dataframe e gravar os dados em um arquivo csv.





In [2]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

**2) Definição de site que utilizarei para pegar os dados**

Escolhi um site que lista os personagens de um jogo mobile, Bungo Stray Dogs - Tales of The Lost, tendo em vista que estes possuem vários atributos interessantes para uma possível análise de dados, detre eles:



1. ID JP: Identificador do personagem na versão japonesa do jogo;
2. ID EN: Identificador do personagem na versão global do jogo;
3. Name: Nome do personagem;
4. Rarity: uma classificação dos atributos de combate do personagem(R, SR, SSR, UR, EX - Essa é a ordem crescente de raridade, quanto maior a raridade, melhores são os atributos de combate do personagem);
5. Affinity: elemento do personagem (Dark, Light, Crimson, Azure e Emerald), importante para definir quais personagens tem vantagens de dano sobre os outros;
6. Gender: gênero do personagem (Male e Female);
7. Affiliation: organização dentro da narrativa do jogo que o personagem participa (Port Mafia, Armed Detective Agency, Guild, Independent, Mimic, Order of the Clock Tower, Rats in the House of the Dead, Supernatural Ability Special Investigation Division);
8. Type: classe do personagem (Balanced, Defensive, Offensive, Special e Support);
9. Release Date JP: data de lançamento na versão japonesa;
10. Release Date EN: data de lançamento na versão global;
11. HP base: valor básico da vida do personagem, antes das melhorias;
12. HP max: valor máximo da vida do personagem ao terminar todas as melhorias;
13. ATK base: valor básico do ataque do personagem, antes das melhorias;
14. ATK max: valor máximo do ataque do personagem ao terminar todas as melhorias;
15. Avaliability: formas de obter o personagem dentro do jogo (Events, Scouts etc)



In [3]:
table_page = str('https://bsdmayoi.fandom.com/wiki/Characters')

**3) Montar lista com os links que serão capturados da tabela resumida de personagens**

In [4]:
#Lista de links para serem utilizados
links = []

#Passar pelo link
page = requests.get(table_page)
soup = BeautifulSoup(page.content, 'html.parser')
characters = soup.find_all(class_ = 'character-entry')

for char in characters:
    # print(f"https://bsdmayoi.fandom.com/{char.find('a')['href']}") #Print para conferência!
    links.append(f"https://bsdmayoi.fandom.com{char.find('a')['href']}") #Montar o formato do link


**4) Definir listas para guardar os valores extraídos do site:**

In [5]:
id_jp_list = []
id_en_list = []
name_list = []

rarity_list = []
affinity_list = []
gender_list = []
affiliation_list = []
type_list = []

release_date_jp_list = []
release_date_en_list = []

hp_base_list = []
atk_base_list = []
hp_max_list = []
atk_max_list = []

avaliability_list = []


**5) Extrair dados de cada página de personagem da lista montada:**

In [6]:
for character_page in links: #Links é a lista de páginas de personagens criadas no tópico 3
    page = requests.get(character_page)

    #Definição do Parser HTML
    soup = BeautifulSoup(page.content, 'html.parser')
    
    #identificadores
    ids = soup.find('div', {'data-source':'no'}).find('div').text #Pega o id no formato "id_jp / id_en"
    id_jp_list.append(ids.split(' / ')[0])
    id_en_list.append(ids.split(' / ')[1])
    name_list.append(soup.find('div', {'data-source':'name'}).find('div').text)

    #Características
    rarity_list.append(soup.find('div', {'data-source':'rarity'}).find('a')['title'])
    affinity_list.append(soup.find('div', {'data-source':'attribute'}).find('a')['title'][9:])
    gender_list.append(soup.find('div', {'data-source':'gender'}).find('div').text)
    affiliation_list.append(soup.find('div', {'data-source':'affiliation'}).find('div').text)
    type_list.append(soup.find('div', {'data-source':'type'}).find('a')['title'][9:])

    #Datas de Lançamento
    release_date_jp_list.append(soup.find('div', {'data-source':'release_date_jp'}).find('div').text)
    release_date_en_list.append(soup.find('div', {'data-source':'release_date_en'}).find('div').text)

    #Atributos de Combate
    hp_base_list.append(soup.find('div', {'data-source':'hp_base'}).text)
    atk_base_list.append(soup.find('div', {'data-source':'atk_base'}).text)
    hp_max_list.append(soup.find('div', {'data-source':'hp_max'}).text)
    atk_max_list.append(soup.find('div', {'data-source':'atk_max'}).text)

#Extrair a disponibilidade do personagem em outra página:
for character_page in links:
    #Forma de obtenção:
    page2 = requests.get(f'{character_page}/Availability') #Basta complementar a página do personagem com /Avaliability
    soup2 = BeautifulSoup(page2.content, 'html.parser')
    test = soup2.find('h2').find('span')
    if test is None:
        avaliability_list.append('Not Avaliable') #Verificar se o elemento existe na estrutura HTML daquela página (em algumas, não tem)
        # print(character_page, 'IS NONE')
    else:
        avaliability_list.append(test.text)
        # print(character_page, test.text)


**6) Montar o dataframe:**

In [7]:
df = pd.DataFrame({
        'id_jp': id_jp_list, 
        'id_en': id_en_list, 
        'name': name_list, 
        'rarity':rarity_list, 
        'affinity': affinity_list, 
        'gender': gender_list, 
        'affiliation': affiliation_list, 
        'type': type_list, 
        'release_date_jp': release_date_jp_list, 
        'release_date_en': release_date_en_list, 
        'hp_base': hp_base_list,
        'hp_max': hp_max_list, 
        'atk_base': atk_base_list,
        'atk_max': atk_max_list,
        'avaliability' : avaliability_list
        })

df #Exibir o dataframe

,id_jp,id_en,name,rarity,affinity,gender,affiliation,type,release_date_jp,release_date_en,hp_base,hp_max,atk_base,atk_max,avaliability
0,0754,0738,Tachihara Michizou,UR,Dark,Male,Port Mafia,Support,2023-05-11 17:00,2023-05-11 08:00,1014,10617,218,2594,Awakening
1,0753,0737,Nakajima Atsushi,SSR,Dark,Male,Armed Detective Agency,Defensive,2023-05-01 14:00,2023-05-01 05:00,1081,10482,192,1456,Scouts
2,0752,0736,Herman M.,SR,Azure,Male,Guild,Support,2023-04-20 17:00,2023-04-20 08:00,676,4511,138,1082,Events
3,0751,0735,Mark T.,SSR,Crimson,Male,Guild,Special,2023-04-20 17:00,2023-04-20 08:00,959,8938,183,1526,Scouts
4,0750,0734,Louisa A.,SSR,Light,Female,Guild,Balanced,2023-04-20 17:00,2023-04-20 08:00,925,8272,189,1690,Scouts
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
749,0005,0005,Nakajima Atsushi,SSR,Crimson,Male,Armed Detective Agency,Offensive,?,?,848,6931,188,2005,Scouts
750,0004,0004,Nakajima Atsushi,SR,Light,Male,Armed Detective Agency,Defensive,?,?,717,5378,130,779,Miscellaneous
751,0003,0003,Nakajima Atsushi,SR,Azure,Male,Armed Detective Agency,Balanced,?,?,683,4754,137,963,Scouts
752,0002,0002,Nakajima Atsushi,SR,Emerald,Male,Armed Detective Agency,Offensive,?,?,649,4130,144,1206,Scouts


**7) Gravar os dados do dataframe para um arquivo csv:**

In [8]:
df.to_csv('Bungo_Data.csv')